In [ ]:
import numpy as np



In [ ]:
for file_path in all_mat_files:
    # load .mat file
    mat_data = scipy.io.loadmat(file_path)

    ## Denoising ECG

    
    Nvar = Nvar_Calculation_from_std_dev(ecg)

    # Saved the denoised ECG file

    ## GEtting the SNOMED code

    ## Figure out how to consolidate the SNOMED code into four classes which will be used for the labelling of the ECG
    #One-hot code the variable

## Figure out how to consolidate all the ECG into a format that can be inputted to the Neural Network

In [ ]:
def load_ecg_for_model(record_path, target_len = 4096):
    """
    Load WFDB ECG record and prepare input for model

    Parameters
    --------
    record_path : str
        Path + record name without extension
        Example: "data/100" for files 100.mat and 100.hea
    target_len: int
        Required length for model input

    Returns
    -----
    ecg: np.ndarray
        Shape(1,target_len,num_leads)
    """

    # Load WFDB record
    record = wfdb.rdrecord(record_path)

    #signal shape: (samples, leads)
    ecg = record.p_signal

    # Gives the number of points that has been recorded by the ECG.
    # If 500 Hz for 10 s will be 5000 points
    n_sample = ecg.shape[0] 

    # Center crop
    start = (n_sample - target_len)//2
    ecg = ecg[start:start+ target_len, :]

    #Add batch dimension
    ecg = np.expand_dims(ecg, axis = 0)

    return ecg

In [ ]:
# Code for processing files sequentially and checking if the denoising works step by step

#--------------------------------------------------------
## Loop though all .mat files and storing their file path
#---------------------------------------------------------

all_mat_files = []
label_rows = []

#os.walk(base_path) returns the three arguments root,dirs,files
#os.walk steps into each folder and then each subfolder until it finds the files
#os.walk uses dirs to know where to go next
for root, dirs,files in os.walk(base_path): 
    for file in files:
        if not file.endswith(".mat"):
            continue
           # all_mat_files.append(os.path.join(root,file))

#print(f"Found {len(all_mat_files)} ECG.mat files")

#----------------------
## Processing each ECG
#----------------------
        print(file)
  
        mat_path = os.path.join(root,file)
        header_path = mat_path.replace(".mat", ".hea")

        #Load ECG
        mat_data = scipy.io.loadmat(mat_path)
        ecg = mat_data['val']
        #print(ecg.shape)

        #plot_ecg_stacked(ecg, fs=500, n_signals=5, lead_names=None,spacing_factor=2, linewidth=1)   
        
        # Use a Butterworth low pass filter
        ecg_low_pass = Butterworth_lowpass_filter(ecg, fs, cutoff, order = 2)
        #print(ecg_low_pass.shape)
        #quick_plot(ecg_low_pass, lead=5, n=1000)
        #plot_ecg_stacked(ecg_low_pass, fs=500, n_signals=5, lead_names=None,spacing_factor=2, linewidth=1)
        
        # Use a robust loess filter
        # Need to process it one lead at a time
        from joblib import Parallel, delayed
        
        ecg_loess = np.array(
            Parallel(n_jobs=-1)(
                delayed(remove_baseline_loess)(ecg_low_pass[ilead,:], fs, frac=0.05, it=3)
                for ilead in range(ecg_low_pass.shape[0])
            )
        )

        
        # ecg_loess = []

        # for ilead in range(ecg_low_pass.shape[0]):
        #     ecg_loess_lead = remove_baseline_loess(ecg_low_pass[ilead,:], fs, frac = 0.05, it =3)
        #     ecg_loess.append(ecg_loess_lead)
        # ecg_loess = np.array(ecg_loess) #Converting it to an array
        
        #print(ecg_low_pass.shape)
        #quick_plot(ecg_low_pass, lead=5, n=1000)
        #plot_ecg_stacked(ecg_loess, fs=500, n_signals=5, lead_names=None,spacing_factor=2, linewidth=1)
        #breakpoint()
                             
        # Use the NLM denoising

        # Nvar is the noise variance, controld the smoothing strength,
        # Weights w = exp(-distance/h) where h = 2*Npatch*Nvar**2
        # if Nvar is too small, almost no denoising, if too large there is morphology distortion
        # Noise_std is estimated from flat segments
        Nvar = Nvar_Calculation_from_std_dev(ecg_loess)

        #ecg_denoised = NLM_1dDarbon_2D_full(ecg_loess, Nvar, P, PatchHW)
        
        ecg_denoised = np.array([
            NLM_1dDarbon(ecg_loess[ilead, :], Nvar, P, PatchHW)
            for ilead in range(ecg_loess.shape[0])
            ])

        # ecg_denoised = []
        # for ilead in range(ecg_low_pass.shape[0]):
        #     ecg_denoised_lead =  NLM_1dDarbon(ecg_loess[ilead,:],Nvar,P,PatchHW)
        #     ecg_denoised.append(ecg_denoised_lead)
        # ecg_denoised = np.array(ecg_denoised) #Converting it to an array

        plot_ecg_stacked(ecg_denoised, fs=500, n_signals=5, lead_names=None,spacing_factor=2, linewidth=1)
        breakpoint()
        
        
        #ecg_denoised = NLM_1dDarbon(ecg_loess,Nvar,P,PatchHW)

        #quick_plot(ecg_low_pass, lead=0, n=5000)
        

        # Get the filename without its extension
        record_id = os.path.splitext(file)[0]

        # save ECG
        np.save(f"CleanedECG/{record_id}.npy",ecg_denoised)

        # Extract Group using SNOWMED labels
        group_code = extract_group(header_path)

        label_rows.append([record_id, "|".join(group_code)])

        #breakpoint()

In [ ]:
def process_in_batches(all_mat_files, batch_size,
                       fs, cutoff, P, PatchHW, save_folder):

    total = len(all_mat_files)

    for start in range(0, total, batch_size):
        end = min(start + batch_size, total)

        print(f"\nProcessing batch {start}:{end}")

        batch_files = all_mat_files[start:end]

        with parallel_backend("loky", n_jobs=16):
            Parallel(verbose=10)(
                delayed(process_ecg_file)(
                    mat_path, fs, cutoff, P, PatchHW, save_folder
                )
                for mat_path in batch_files
            )


In [ ]:
# # --------------------------
# # Function to process one file
# # --------------------------
# def process_ecg_file(mat_path, fs, cutoff, P, PatchHW):
#     record_id = os.path.splitext(os.path.basename(mat_path))[0]
#     output_file = os.path.join(save_folder, f"{record_id}.npy")

#     # Skip if already processed
#     if os.path.exists(output_file):
#         print("Skipping:", record_id)
#         return None

#     print("Processing:", record_id)

#     header_path = mat_path.replace(".mat", ".hea")

#     # --------------------------
#     # Load ECG
#     # --------------------------
#     mat_data = scipy.io.loadmat(mat_path)
#     ecg = mat_data['val']  # shape (n_leads, n_samples)

#     # --------------------------
#     # Butterworth low-pass filter
#     # --------------------------
#     ecg_low_pass = Butterworth_lowpass_filter(ecg, fs, cutoff, order=2)

#     # --------------------------
#     # LOESS baseline removal (threading-safe)
#     # --------------------------
#     n_leads = ecg_low_pass.shape[0]

    
#     ecg_loess = np.array([
#         remove_baseline_loess(ecg_low_pass[ilead, :], fs, frac=0.05, it=3)
#         for ilead in range(n_leads)
#     ])


#     # --------------------------
#     # Compute noise variance for NLM
#     # --------------------------
#     Nvar = Nvar_Calculation_from_std_dev(ecg_loess)

#     # --------------------------
#     # NLM denoising (non-vectorized per lead)
#     # --------------------------
#     ecg_denoised = np.array([
#         NLM_1dDarbon(ecg_loess[ilead, :], Nvar, P, PatchHW)
#         for ilead in range(n_leads)
#     ])

#     # plot_ecg_stacked(ecg_denoised, fs=500, n_signals=5, lead_names=None,spacing_factor=2, linewidth=1)
#     # fig = plt.gcf()
#     # fig.savefig(output_plot_path)
#     # plt.close(fig)
    
#     # --------------------------
#     # Save denoised ECG
#     # --------------------------
#     os.makedirs(save_folder, exist_ok=True)
    
#     np.save(os.path.join(save_folder, f"{record_id}.npy"), ecg_denoised)
#     print("Saved:", record_id)
    
#     # --------------------------
#     # Extract grouedp labels
#     # --------------------------
#     group_code = extract_group(header_path)
#     label_row = [record_id, "|".join(group_code)]

#     return label_row

# # --------------------------
# # Main function to process all .mat files
# # --------------------------
# def process_all_ecg(base_path, fs, cutoff, P, PatchHW):
#     # Gather all .mat files recursively
#     print(base_path)
#     all_mat_files = [
#         os.path.join(root, f)
#         for root, dirs, files in os.walk(base_path)
#         for f in files
#         if f.endswith(".mat")
#     ]

#     print(f"Found {len(all_mat_files)} .mat files")

#     test_files = all_mat_files[100:110]
    
#     with parallel_backend("loky",n_jobs=16):
#         label_rows = Parallel(verbose=10)(
#             delayed(process_ecg_file)(mat_path, fs, cutoff, P, PatchHW)
#             for mat_path in test_files
#     )

#     # Process files in parallel (threading backend for debugger-safe)
#     # with parallel_backend('threading', n_jobs=-1):
#     #     label_rows = Parallel()(
#     #         delayed(process_ecg_file)(mat_path, fs, cutoff, P, PatchHW)
#     #         for mat_path in all_mat_files
#     #     )

#     # label_rows = [
#     #     process_ecg_file(mat_path, fs, cutoff, P, PatchHW)
#     #     for mat_path in all_mat_files[100:105]
#     # ]
    
#     return label_rows

# # --------------------------
# # Usage
# # --------------------------
# if __name__ == "__main__":
#     base_path =  "/home/lewisa13/ECG_Data/physionet.org/files/ecg-arrhythmia/1.0.0/WFDBRecords"
#     save_folder = "/home/lewisa13/CleanedECG"
#     output_plot_path = "/home/lewisa13/CleanedECGPlots"
#     fs = 500          # sampling frequency
#     cutoff = 40       # low-pass cutoff
#     P = 20            # NLM search window
#     PatchHW = 3       # NLM patch half-width

#     import time

#     start = time.time()

#     labels = process_all_ecg(base_path, fs, cutoff, P, PatchHW)

#     end = time.time()

#     runtime = end - start
#     print(f"Total runtime: {runtime:.1f} s ({runtime/60:.1f} min)")
#     %print("All files processed. Label rows:", labels)
